# 📘 Week 4 Day 3: 哈希表与数据管道 (Hash Maps & Data Pipelines)

**今日主题**: **查找与索引 (Hash Tables & Data Pipelines)**。

  * **算法侧**: **哈希表 (Hash Map/Dictionary)**。这是计算机科学中“空间换时间”的巅峰之作，也是面试中出现频率最高的数据结构（没有之一）。
  * **PyTorch/AI 侧**: **键值对与数据索引**。
      * 模型保存的本质 (`state_dict`) 就是一个巨大的字典。
      * NLP 中的词表 (Vocabulary) 也就是 Word 到 ID 的哈希映射。
      * **Pandas** 是处理非结构化数据（如 CSV 里的文本）的前置技能，为 Stage 2 做准备。

## 1 核心概念：O(1) 的魔法

**底层联系 (The Connection):**

  * **LeetCode**: 为什么要用 `dict` 或 `set`？为了把查找时间从 $O(N)$ 降到 $O(1)$。
  * **AI 工程**:
      * **权重加载**: 当你调用 `model.load_state_dict()` 时，PyTorch 就是通过字符串匹配（Key: "layer1.weight"）瞬间找到对应的张量。
      * **Tokenization**: BERT 把单词 "Apple" 变成数字 `1034`，查的就是一个巨大的 Hash Map。

## 2 AI 工程实战 (The "Job" Skills)

### 2.1 深入 state\_dict (模型的字典本质)

我们在 Week 3 保存了 `best_model.pth`，它到底是什么？


In [ ]:
import torch
import torch.nn as nn

# 定义一个简单的网络
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 1)

model = SimpleNet()

# 1. 查看 state_dict
# 这是一个 OrderedDict (有序字典)
params = model.state_dict()

print("--- 模型内部的 Hash Map ---")
for key, value in params.items():
    print(f"Key: {key} | Value Shape: {value.shape}")

# 2. 模拟权重加载
# 假设我们只保存了这个字典
torch.save(params, "temp_model.pth")

# 加载回来其实就是字典匹配
loaded_params = torch.load("temp_model.pth")
model.load_state_dict(loaded_params)
print("\n权重加载成功!")

--- 模型内部的 Hash Map ---
Key: fc1.weight | Value Shape: torch.Size([5, 10])
Key: fc1.bias | Value Shape: torch.Size([5])
Key: fc2.weight | Value Shape: torch.Size([1, 5])
Key: fc2.bias | Value Shape: torch.Size([1])

权重加载成功!


/tmp/ipykernel_7056/2895851430.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_params = torch.load("temp_model.pth")


> **💡 面试/实战考点**: 如果你修改了模型结构（比如改了层名），加载权重时会报错 `Missing key(s)` 或 `Unexpected key(s)`。这时你需要手动处理这个字典（比如改 Key 的名字）再 load。

### 2.2 Pandas 预热 (为 Stage 2 处理文本数据做准备)

在 MNIST 中我们用 `torchvision` 自动处理了图片。但在 Stage 2 (Transformers) 中，你面对的通常是 **CSV 文件**（一列 text，一列 label）。

In [4]:
import pandas as pd
import numpy as np

# 1. 模拟一个简单的文本分类数据集
data = {
    'text': ['I love AI', 'Python is great', 'Machine Learning', 'Bug everywhere'],
    'label': [1, 1, 1, 0]
}
df = pd.DataFrame(data)

# 2. 核心操作：Locating (查找)
# Pandas 的查找底层优化非常依赖 Hash 索引
print("--- 筛选 label 为 1 的数据 ---")
positive_samples = df[df['label'] == 1]
print(positive_samples)

# 3. 映射 (Mapping) - 这一步在 NLP 中就是 Tokenization 的原型
vocab = {'I': 1, 'love': 2, 'AI': 3, 'Python': 4, 'is': 5, 'great': 6}

def simple_tokenize(sentence):
    return [vocab.get(word, 0) for word in sentence.split()]

# 使用 apply 高效处理整列数据
df['token_ids'] = df['text'].apply(simple_tokenize)
print("\n--- Tokenization 后的 DataFrame ---")
print(df)

--- 筛选 label 为 1 的数据 ---
               text  label
0         I love AI      1
1   Python is great      1
2  Machine Learning      1

--- Tokenization 后的 DataFrame ---
               text  label  token_ids
0         I love AI      1  [1, 2, 3]
1   Python is great      1  [4, 5, 6]
2  Machine Learning      1     [0, 0]
3    Bug everywhere      0     [0, 0]


## 3 LeetCode 算法突击 (The "Interview" Skills)

哈希表题目通常不难，核心在于**想得起来用**。只要看到**查找、出现次数、配对**，第一时间反应：Hash Map。

### 3.1 两数之和 (Two Sum) - Easy (必考)

  * **LeetCode 链接**: [1. Two Sum](../../LeetCode%20practice/1-50.ipynb)
  * **核心思路**: 边遍历边查。我需要 `target - current_val`，我在字典里找找之前有没有存过这个差值。

In [ ]:
from typing import List

class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        hashmap = {}
        for i, num in enumerate(nums):
            complement = target - num
            if complement in hashmap:
                return [hashmap[complement], i]
            hashmap[num] = i
        return []
  
solution = Solution()

# 手动输入数组
nums_input = input("请输入数组（用逗号分隔）: ")
nums = list(map(int, nums_input.split(',')))  # 将输入的字符串转成整数列表
target = int(input("请输入目标值: "))

print("结果:", solution.twoSum(nums, target))

### 3.2 有效的字母异位词 (Valid Anagram) - Easy

  * **LeetCode 链接**: [242. Valid Anagram](../../LeetCode%20practice/201-250.ipynb)
  * **AI 关联**: **词袋模型 (Bag of Words)**。不考虑顺序，只考虑词频（Word Frequency）。
  * **核心思路**: 统计两个字符串中每个字符出现的次数是否相等。

In [ ]:
class Solution:
    def isAnagram(self, s: str, t: str) -> bool:
        record = [0] * 26
        for i in s:
            record[ord(i) - ord("a")] += 1
        for i in t:
            record[ord(i) - ord("a")] -= 1
        for i in range(26):
            if record[i] != 0:
                return False
        return True

### 3.3 字母异位词分组 (Group Anagrams) - Medium (进阶)

  * **LeetCode 链接**: [49. Group Anagrams](../../LeetCode%20practice/1-50.ipynb)
  * **核心思路**: **Key 的设计**。如何让 "eat", "tea", "ate" 拥有同一个 Key？
      * 方案：将字符串排序作为 Key。`sorted('eat')` -\> `'aet'`。

In [6]:
from typing import List

class Solution:
    def groupAnagrams(self, strs: List[str]) -> List[List[str]]:
        ans = {}
        for s in strs:
            key = ''.join(sorted(s))
            if key in ans:
                ans[key].append(s)
            else:
                ans[key] = [s]  # 初始化为包含当前字符串的列表
        return list(ans.values())
    
# --- 交互式测试 ---
solution = Solution()

print("=== 测试 Group Anagrams ===")
# 提示用户输入
raw_input = input("请输入一组单词 (用逗号分隔，例如: eat, tea, tan): ")

# 处理输入：分割并去除每个单词前后的空格
if not raw_input:
    strs = []
else:
    strs = [s.strip() for s in raw_input.split(',')]

print(f"\n输入的列表: {strs}")
result = solution.groupAnagrams(strs)

print("分组结果:")
for group in result:
    print(group)

=== 测试 Group Anagrams ===

输入的列表: ['eat', 'tea', 'tan']
分组结果:
['eat', 'tea']
['tan']


## 4 总结与作业

### 今日复盘

1.  **AI 工程**:
      * 理解了 `state_dict` 只是一个普通的有序字典。
      * Pandas 的 `apply` 配合字典映射，是处理 NLP 数据最高效的方式之一。
2.  **算法逻辑**:
      * **空间换时间**：通过消耗内存建立 Hash Map，消除嵌套循环，把 $O(N^2)$ 降为 $O(N)$。

### 课后作业

1.  **运行代码**: 在 Notebook 中跑通所有示例。
2.  **思考/实操**:
      * 打开你的 `src/models/simple_cnn.py`，看看你自己定义的层名（比如 `self.conv1`）。
      * 想象一下，如果我现在给你一个训练好的模型文件，但是里面的 Key 是 `layer1` 而你的代码里叫 `conv1`，你会怎么写一个脚本把 Key 改名并加载进去？（这是非常真实的 Transfer Learning 场景）。